# Import

## Import Library

In [12]:
import json
import numpy as np

import nltk
from nltk.stem.porter import PorterStemmer

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

## Define Variable

In [51]:
PATH_INTENTS = '../data/intents_3.json'
PATH_MODEL = '../model/model_nn.h5'
PATH_METADATA = '../model/meta/metadata_nn.json'
PATH_DATA = '../model/meta/data_nn.pth'

In [14]:
# Define list
all_words = []  # Setence
tags = []       # Tags of setence
xy = []         # X and Y of setence

## Create NLP Utils

In [15]:
nltk.download('punkt')
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WINDOWS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
def tokenize(setence):
    return nltk.word_tokenize(setence)

In [17]:
def stem(word):
    return stemmer.stem(word.lower())

In [18]:
def bag_of_words(tokenized_setence, all_words):
    # Example
    """
    setence = ['hello', 'how', 'are', 'you']
    words   = ['hi', 'hello', 'I', 'you', 'bye', 'thank', 'cool']   -> all words from pattern
    bog     = [  0 ,     1  ,  0 ,   1  ,   0  ,    0   ,    0  ]   -> result of bag of word
    """
    
    # Stemming
    tokenized_setence = [stem(w) for w in tokenized_setence]
    
    # Make zero value array
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        # When word there in tokenized_setence
        if w in tokenized_setence:
            # Change value from 0 to 1
            bag[idx] = 1.0
    
    # Return the result bag of word
    return bag

## Read Dataset

In [19]:
# Open file json
with open(PATH_INTENTS, 'r') as f:
    intents = json.load(f)

In [20]:
# Getting setence, tags, xy
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

# Data Preparation

## NLP Preprocessing

In [21]:
# Stemming
ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words] 

In [22]:
# Remove duplicate word
all_words = sorted(set(all_words))      
tags = sorted(set(tags)) 
# print(tags)

In [23]:
# Define train data
x_train = []
y_train = []
for (pattern_setence, tag) in xy:
    # Getting the feature
    bag = bag_of_words(pattern_setence, all_words)
    x_train.append(bag)
    
    # Getting label from index of tag
    label = tags.index(tag)
    y_train.append(label)   # CrossEntropyLoss

In [24]:
# Convert to array
x_train = np.array(x_train)
y_train = np.array(y_train)

## Create Dataset

In [25]:
# Create custom class dataset
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

In [26]:
# Create dataset
dataset = ChatDataset()
# train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)
# train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers = 2, persistent_workers=True)

In [27]:
for (words, labels) in train_loader:
    first_word = words
    first_label = labels
    print(f'words \t: {words}')
    print(f'labels \t: {labels}')
    if first_word != None and first_label != None:
        break

words 	: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])
labels 	: tensor([23, 39, 92, 24, 58, 72, 13,  2], dtype=torch.int32)


In [28]:
all_words[0:5]

["'d", "'ll", "'m", "'re", "'s"]

In [29]:
xy[0:3]

[(['Hi'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting')]

In [30]:
len(tags)

100

In [31]:
first_word.shape

torch.Size([8, 529])

In [32]:
first_label

tensor([23, 39, 92, 24, 58, 72, 13,  2], dtype=torch.int32)

In [33]:
x_train.shape

(668, 529)

In [34]:
y_train.shape

(668,)

# Modeling

## Define Hyperparameter

In [39]:
# Define Hyperparameter
batch_size = 8
hidden_size = 8
input_size = len(x_train[0])
output_size = len(tags)
learning_rate = 0.001
num_epochs = 1000
# print(input_size, len(all_words))
# print(output_size, tags)

In [40]:
# Checking cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
torch.cuda.is_available()

False

## Create Model

In [42]:
# Create Feed Forward Neural Network Algorithm
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # First linear layer
        out = self.l1(x)
        out = self.relu(out)
        
        # Second linear layer
        out = self.l2(out)
        out = self.relu(out)
        
        # Third linear layer
        out = self.l3(out)    # no activation and no softmax
        return out

In [43]:
# Create model
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [44]:
# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training Model

In [45]:
# Training model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device, dtype=torch.int64)
        
        # Forward
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # Backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Print loss per 100 epoch
    if (epoch + 1) % 100 == 0:
        print(f'epoch : {epoch+1}/{num_epochs}, loss={loss.item():.4f}')

# Print final loss when training done
print(f'final loss, loss={loss.item():.4f}')

epoch : 100/1000, loss=0.0941
epoch : 200/1000, loss=0.0041
epoch : 300/1000, loss=0.0004
epoch : 400/1000, loss=0.0000
epoch : 500/1000, loss=0.0000
epoch : 600/1000, loss=0.2497
epoch : 700/1000, loss=0.0000
epoch : 800/1000, loss=0.0000
epoch : 900/1000, loss=0.0000
epoch : 1000/1000, loss=0.0000
final loss, loss=0.0000


# Evaluation

## Loss Model

In [46]:
# Print final loss when training done
print(f'final loss, loss={loss.item():.4f}')

final loss, loss=0.0000


## Save Model

In [52]:
# Save data
data = {
    'model_state' : model.state_dict(),
    'input_size' : input_size,
    'output_size' : output_size,
    'hidden_size' : hidden_size,
    'all_words' : all_words,
    'tags' : tags,
}

# Save file of data
torch.save(data, PATH_DATA)
print(f'Training complete. file saved to {PATH_DATA}')

Training complete. file saved to ../model/meta/data_nn.pth
